## 1.  Import libraries 

In [33]:
import re 
import json

## 2. Parse 29628792.dat File

First of all i want to read the data file into a python variable.

In [34]:
data = open('29628792.dat','r')

Then i want to differentiate every job advirtesment. These advirtesements are seperated by successive mineses (-).
Therefore, i can use split() function on data variable after changing it into a string because it is a .TextIOWrapper type variable.

To do that i can use readlines() and get every single line and save them into a list of strings then i can split this list using entries that are consist of just successive mineses.

In [35]:
new_data=data.readlines()
        

In [36]:
len(new_data)

1623590

Now, we know that there are 1,623,590 lines in our data file. And every job advertisement are seperated with each other by a line that are consist of only successive minuses (-).
        Therefore, we can concatanate lines between minuse signs then we will have list of strings that every string represents a job advertisement.

In [37]:
jobs = []
string=''

for lines in new_data:
    if lines=='------------------------------\n':
        jobs.append(string)
        string=''
    else:
        string += lines

In [38]:
len(jobs)

32764

Now we have a list with 32,764 strings inside it and every string represents a job listing.

I am expecting to have 11 different attributes in every job advertisment.( If the dataset doesn't have missing information or nulls in it.)

These are;
* Listing ID
* Job Title
* Location
* Application Deadline
* Start Date
* Company Information
* Application Procedure
* Salary
* Needed Qualifications
* Job Responsibilities
* Listing Description


To capture these attributes, we need to use regular expression that search for the expressions that are located at the begininning of the line( which means start with /n) and end with : and space. (There can be exceptions but i think this is a good point to start with.)

 Additionaly, we know that there are 11 attributes so if there are more or less atrributes in any job posting, it can be a hint for me to understand exceptions and NA's.

Therefore, i will write a for loop to take every job posting one by one and capture attributes then create a dictionary with these attributes and their corresponding value.

In [39]:
sample = jobs

In [40]:
keys=[]
pattern=re.compile(r"^[\w]+[ ]?[\w]*:")


for each in sample:
    a=re.findall(r"^[\w]+[ ]?[\w]*:",each,re.M)
    keys.append(a)
    


    


    

    

The regex used above matches with expressions that starts with any alphanumeric character or an underscore then may have a space and then may have zero or more alphanumeric character or underscore.

Now we have the attributes of every job posting but we have extra keys which are not really an attribute. In order to get ride of these extra keys, i will consider to write unique regular expression for every attribute using key words for every attribute. For example for company information, there should be (comp, info, about, company, inf ... ) so i can use this kind of key words for all attributes and if there is a key that doesn't fits in any of these keywords i can drop it.

* Listing ID      - Must include [id] 
* Job Title   - Must include one of ['ttl' , 'title'] or start with job then _ or space then ['t''i''t''e'] one to 4 of them.
* Location  - Must include 'loc' or start with 'address' and letters then ',' letters and ',' again.
* Application Deadline  - Must include 'dead' or start with 'app' then [lications] one to all of them then there may be an _ or space then a 'd'.
* Start Date - Must include one of ['start','strt'] or start with 's' then _ or space then a 'd'. Another option is just 'dates'.
* Company Information - Must include one of ['inf','comp','about']
* Application Procedure  - Must include one of ['prc','pro'] or start with 'app' then [lications] one to all of them then there may be an _ or space then a 'p'.
* Salary - Must include one of ['sal','rem']
* Needed Qualifications - Must include one of ['req','qua','needed']
* Job Responsibilities - Must include 'resp'
* Listing Description = Must include 'des'

In [41]:
id_capture = r"id"
title_capture = r"titl|ttl|title|job[_ ]*[tite]{1,4}"
loc_capture = r"loc[ation]{0,5}|address[: ][/w]+,[/w]+,"
deadline_capture = r"dead|app[lications]{0,9}[_ ]*d"
start_capture = r"start|strt|s[_ ]*d|dates"
info_capture = r"inf|comp|about"
procedure_capture = r"prc|pro|app[lications]{0,9}[_ ]*p"
salary_capture = r"sal|rem"
qual_capture = r"req|qua|needed"
resp_capture = r"resp"
desc_capture = r"des"

In [42]:

for job in range(len(keys)):
    for attr in range(len(keys[job])-1,-1,-1):
        
        
        if re.search(id_capture,keys[job][attr],re.I) != None :
            continue
        
        
        elif re.search(title_capture,keys[job][attr],re.I) != None:
            continue
        
        
        elif re.search(loc_capture,keys[job][attr],re.I) != None:
            continue
        
        
        elif re.search(deadline_capture,keys[job][attr],re.I) != None:
            continue
        
        
        elif re.search(start_capture,keys[job][attr],re.I) != None:
            continue
            
        
        elif re.search(info_capture,keys[job][attr],re.I) != None:
            continue
        
        
        elif re.search(procedure_capture,keys[job][attr],re.I) != None:
            continue
        
        
        elif re.search(salary_capture,keys[job][attr],re.I) != None:
            continue
        
        
        elif re.search(qual_capture,keys[job][attr],re.I) != None:
            continue
            
        
        elif re.search(resp_capture,keys[job][attr],re.I) != None:
            continue
            
        
        elif re.search(desc_capture,keys[job][attr],re.I) != None:
            continue
            

        
        del keys[job][attr]


        
        


Now we have got rid of wrong attributes and ready to find the values of attributes and create a list of dictionaries including every job posting as an element of list. The regular expressions below captures the values of every attribute.

In [43]:
values=[]

for i in range(len(keys)):
    temporary_list=[]
    for j in range(len(keys[i])-1):
        temporary_list.append(re.findall(r"%s(.*?)\n%s" %(keys[i][j],keys[i][j+1]) , sample[i],re.S))
    
    temporary_list.append(re.findall(r"%s(.*)"%keys[i][j+1], sample[i], re.M ))
    
    values.append(temporary_list)

The chunk of code below is just to make sure that there are exactly the same number of attributes and their corresponding values.

In [44]:
for i in range(len(keys)):
    if len(keys[i]) != len(values[i]):
        print(i)

Now we have 2 lists of strings, one of them has the attributes and the other one has the corresponding values of the attributes. ( Both of them are in the same order! )

Next step is to change the attribute names to make them steady for all job postings and labelling the missing values as N/A.

To do that, i will use the same regexes that i used above to capture all atributes one by one.

In [45]:
for job in range(len(keys)):
    for attr in range(len(keys[job])-1,-1,-1):
        
        
        if re.search(id_capture,keys[job][attr],re.I) != None :
            keys[job][attr]='ID'
        
        
        elif re.search(title_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Title'
        
        
        elif re.search(loc_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Location'
        
        
        elif re.search(deadline_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Application_Deadline'
        
        
        elif re.search(start_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Start_Date'
            
        
        elif re.search(info_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='About_Company'
        
        
        elif re.search(procedure_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Application_Procedure'
        
        
        elif re.search(salary_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Salary'
        
        
        elif re.search(qual_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Qualifications'
            
        
        elif re.search(resp_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Responsibility'
            
        
        elif re.search(desc_capture,keys[job][attr],re.I) != None:
            keys[job][attr]='Description'

Now i need to get rid of repeating attributes because my regex may catch more than one times the same attribute and there can be repeating attributes in the dataset as well. I can do it just by changing these 2 lists into a list of dictionaries and if there is a repeating attribute ( which will be a key value in dictionaries) the second appeareance of it will be taken into account and first one will be deleted because my loop assigns the last value with the same key below.

In [46]:
list_of_dict=[]
for i in range(len(keys)):
    temporary_dict={}
    for j in range(len(keys[i])):
        temporary_dict[keys[i][j]]=values[i][j]
    
    list_of_dict.append(temporary_dict)
    
    

Now, i need to deal with the N/A's. There are some job postings that has missing values which means in dictionary of each posting that has key values less than 11 , has a missing key value.

Therefore, i will find the missing key values in every dictionary and add these keys and their corresponding value which is NA.

In [47]:
for i in range(len(list_of_dict)):
    
    all_attr=['ID', 'About_Company', 'Description', 'Responsibility', 'Qualifications', 'Application_Deadline', 'Title', 'Application_Procedure', 'Salary', 'Location', 'Start_Date']
    
    for j in list_of_dict[i]:
        a=all_attr.index(j)
        del all_attr[a]
    
    if len(all_attr) !=0 :
        for k in all_attr:
            list_of_dict[i][k]=['NA']
        
        
            

Now i will check is there a repeating ID's and if there is i will delete the whole job posting of the second appearance of repeating ID in the List of dictionary structure.

In [48]:
ids=[]
del list_of_dict[0]
for job in range(0,len(list_of_dict)):
    ids.append(list_of_dict[job]['ID'][0])



ids=set(ids)
    

In [49]:
len(list_of_dict)

32763

In [50]:
for job in range(len(list_of_dict)-1,-1,-1):
    if list_of_dict[job]['ID'][0] in ids:
        ids.remove(list_of_dict[job]['ID'][0])
        
    elif list_of_dict[job]['ID'][0] not in ids:
        del list_of_dict[job]
        

In [51]:
len(list_of_dict)

26805

By looking at length of list_of_dict variable whose every entry (dictionary) represents a job posting, there were about 6000 repeating ID's in dataset.

Using the chunk of code below, i got ride of the new line characters to make the content of the structure appropriate and more readable.

In [52]:
all_attr=['ID', 'About_Company', 'Description', 'Responsibility', 'Qualifications', 'Application_Deadline', 'Title', 'Application_Procedure', 'Salary', 'Location', 'Start_Date']

for job in range(len(list_of_dict)):
    for att in all_attr:
        list_of_dict[job][att]=list_of_dict[job][att][0].strip()
        list_of_dict[job][att]=list_of_dict[job][att].replace('\\\n',' ')
        list_of_dict[job][att]=list_of_dict[job][att].replace('\\n',' ')
        list_of_dict[job][att]=list_of_dict[job][att].replace('\n',' ')
        
        
            

 ## 3. Changing into JSON and Creating Output File

In [53]:
new_list = []
dictionary = {}


for job in list_of_dict:
    dictionary['listing']=job
    new_list.append(dictionary)

In [54]:
final={}
final['listings']=new_list


In [55]:
with open('29628792.json', 'w') as output:
    json.dump(final,output, indent=4)

  ## 4. Changing into XML and Creating Output File

In [56]:
xml='<?xml version="1.0" encoding="UTF-8" ?> \n<listings>'

for job in list_of_dict:
    xml+='\t<listing id=' +"'" + job['ID'] + "'>" + '\n' +'\t\t'
    xml+= '<title>' + job['Title'] + '</title>' + '\n' +'\t\t'
    xml+= '<location>' + job['Location'] + '</location>' + '\n' +'\t\t'
    xml+= '<job_descriptions>' + '\n' +'\t\t\t' 
    xml+= '<description>' + job['Description'] + '</description>' +'\n' +'\t\t'
    xml+= '</job_descriptions>' + '\n' + '\t\t'
    xml+= '<job_responsibilities>' + '\n' +'\t\t\t' 
    xml+= '<responsibility>' + job['Responsibility'] + '</responsibility>' + '\n' + '\t\t'
    xml+= '</job_responsibilities>' + '\n' + '\t\t'
    xml+= '<required_qualifications>' + '\n' +'\t\t\t'
    xml+= '<qualification>'  + job['Qualifications'] + '</qualification>' + '\n' + '\t\t'
    xml+= '</required_qualifications>'  + '\n' + '\t\t'
    xml+= '<salary>' + job['Salary'] + '</salary>'  + '\n' + '\t\t'
    xml+= '<application_procedure>' + job['Application_Procedure'] + '</application_procedure>' + '\n' + '\t\t'
    xml+= '<start_date>' + job['Start_Date'] + '<start_date>' + '\n' + '\t\t'
    xml+= '<application_deadline>' +job['Application_Deadline'] +'</application_deadline>' + '\n' + '\t\t'
    xml+= '<about_company>' + job['About_Company'] + '</about_company>' + '\n' +'\t'
    xml+= '</listing>'

    
xml+='\n</listings>'
    

In [57]:
with open('29628792.xml', 'w') as output2:
    output2.write(xml)